<a href="https://colab.research.google.com/github/seraogianluca/mircv-exercises/blob/main/Exercise1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#MIRCV 2021
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install whoosh

In [ ]:
# %matplotlib inline

from whoosh import index
from whoosh.fields import *
from whoosh.analysis import *
from whoosh import qparser
import os, os.path

from matplotlib import pyplot as plt
from IPython.display import display, HTML

from base64 import b64encode
from urllib.parse import quote
from io import BytesIO

BASE_DIR = '/content/gdrive/My Drive/mircv2021/'

COCO_CAPTIONS_DATASET = 'data/txt/captions.properties'
COCO_ANNOTATIONS_DATASET = 'data/txt/annotations.properties'
INDEX_NAME = 'coco'
IMG_DIR = BASE_DIR + 'data/' + 'coco_img'

In [ ]:
#TODO Indexing

if not os.path.exists(BASE_DIR + "out"):
    os.mkdir(BASE_DIR + "out")
if not os.path.exists(BASE_DIR + "out/whoosh"):
    os.mkdir(BASE_DIR + "out/whoosh")

#create whoosh index
schema = Schema(annotations=TEXT(analyzer=KeywordAnalyzer(), vector=True, stored=True), 
                text=TEXT(analyzer=KeywordAnalyzer(), vector=True, stored=True),
                id=ID(stored=True))
ix = index.create_in(BASE_DIR + 'out/whoosh', schema, indexname=INDEX_NAME)
print('creating ' + INDEX_NAME + ' index...')

#initialize whoosh writer
writer = ix.writer()

#index all the caption text
with open(BASE_DIR + COCO_CAPTIONS_DATASET, 'r') as captions:
  for line in captions:
    caption = line.split('=')

    with open(BASE_DIR + COCO_ANNOTATIONS_DATASET, 'r') as annotations:
      for ln in annotations:
        annotation = ln.split('=')
        if (annotation[0] == caption[0]):
          writer.add_document(annotations= annotation[1], text=caption[1], id=caption[0])
      
#commit writer
writer.commit()

print('indexing done ')


creating coco index...
indexing done 


In [ ]:
def get_table(img, title, txt):
    table ='<table><td><img style="width:320px" src="' + img + '"></td>'
    table +='<td style="text-align:left"><b>' + title + '</b></p>' + txt + '</td></table>'
    return table


def display_result(img_path, title, txt):
    with open(img_path, 'rb') as f:
        dataEncoded = b64encode(BytesIO(f.read()).getvalue())
        data = 'data:image/jpg;base64,{}'.format(quote(dataEncoded))
        display(HTML(get_table(data, title, txt)))

In [ ]:
#TODO Searching

#txt = "annotations:person text:people text:bike"
#txt = "annotations:person text:people bike"

#txt = "annotations:person text:people NOT text:bike"
#txt = "annotations:person text:beach OR text:bird"
#txt = "annotations:person text:people text:bike"
txt = "annotations:person text:beach OR text:bird"

print("query: " + txt)

#initialize whoosh searcher
searcher = ix.searcher()

#perform txt query
parser = qparser.QueryParser('text', ix.schema, group=qparser.AndGroup)
txt_query = parser.parse(txt)

#display the search results
res = searcher.search(txt_query)
print(res)

for hit in res:
  score = hit.score
  id = hit['id']
  text = hit['text']
  display_result(IMG_DIR + '/' + id.zfill(12) + '.jpg', id, text)
  
  #without HTML 
  #print("Id: ", id, " score: ", score)
  #img = plt.imread(IMG_DIR + '/' + id.zfill(12) + '.jpg')
  #plt.imshow(img)
  #plt.show()
  #print("text: ", text)

query: annotations:person text:beach OR text:bird
<Top 10 Results for And([Term('annotations', 'person'), Or([Term('text', 'beach'), Term('text', 'bird')])]) runtime=0.0038492889998451574>
